# **Music Store Data Analysis Using SQL**

### We are provided a database of a Music Store and some questions that could be solved with a query in SQL.

### Question 1 - Who is the senior most employee based on the job title?





In [ ]:
select *
from employee
order by levels desc
limit 1

In [ ]:
#Answer - We have the table named employee which holds the information about all the employees.
#We select the information from employee and to order the table by levels
#which is respective to employees and their seniority in the company.

### Question 2 - Which countries have the most invoices?

In [ ]:
select billing_country, count(billing_country)
from invoice
group by billing_country
order by count desc

In [ ]:
#Answer - After checking invoice table we saw that there are records of countries with their
#respective invoices but we had to use count and greoup by sql queries to
#get the countries in the order that had the most invoices.

### Question 3 - What are the top 3 values of total invoices?

In [ ]:
select *
from invoice
order by total desc
limit 3

In [ ]:
#Answer - So for top 3 invoices we had to sort the table in descending order by total
#column name and limiting the output to 3 by using limit function.

### Question 4 - Which city has the best customers? We would like to throw a promotional Music Festival in the city we made the most money. Write a query that returns one city that has the highest sum of invoice totals. Return both the city name & sum of all invoice totals.


In [ ]:
select billing_city, sum(total)
from invoice
group by billing_city
order by sum desc
limit 1

In [ ]:
#Answer - For finding best customers, we have to select the city and sum of the
#total money spent by the customers in the respective cities and then ordering it in
#descending order to find the city where customers purchased maximum music albums.

### Question 5 - Who is the best customer? The customer who has spent the most money will be declared the best customer. Write a query that returns the person who has spent the most money.


In [ ]:
select customer_id, first_name, last_name, sum(total) as TotalSpent
from customer
join invoice
using(customer_id)
group by customer_id, first_name, last_name
order by totalspent desc
limit 1

In [ ]:
#Answer - To find out the customer who have spent the most money on albums and is beneficial to business,
#we have to check through the customer table and invoice table, to see if there are matching values
#because data of cutomers and money spent both are in different tables,
#so we used inner join to filter out rest of the information and used this query to find the top customer.

### Question 6 -  Write a query to return the email, first name, last name of all Rock Music listeners. Return your list ordered alphabetically by email starting with A.


In [ ]:
select distinct email, first_name, last_name
from customer
join invoice using(customer_id)
join invoice_line using(invoice_id)
join track using(track_id)
join genre using(genre_id)
where genre_id = '1'
order by email

In [ ]:
#Answer - So to get the information of all rock listeners we need to go step by step joining
#tables with same ids or any column that is similar, so first we used customer_id to
#join custimer and invoice then invoice_id  to join further and so on we got the part where we
#extracted the customers who listen to rock music with the genre_id.

### Question 7 - Let's invite the artists who have written the most rock music in our dataset. Write a query that returns the Artist name and total track count of the top 10 rock bands.


In [ ]:
select artist.name, count(track.name) as track_count
from artist
join album using(artist_id)
join track using(album_id)
where genre_id = '1'
group by artist.name
order by track_count desc
limit 10

In [ ]:
#Answer -  We have to go through artist table and make our way through album and then track using columns that are same in them.

### Question 8 -  Return all the track names that have a song length longer than the average song length. Return the Name and Milliseconds for each track. Order by the song length with the longest songs listed first.

In [ ]:
select name, milliseconds
from track
where milliseconds >
(select avg(milliseconds)
 from track)
order by milliseconds desc

In [ ]:
#Answer - To return milliseconds greater than avergae we use a subquery inside our main to compare the lengths in where statement.

### Question 9 -  Find how much amount spent by each customer on artists? Write a query to return customer name, artist name and total spent.

In [ ]:
WITH CTE AS (
SELECT artist.artist_id, artist.name, sum(invoice_line.unit_price * invoice_line.quantity) as sales
	from artist
	join album using(artist_id)
	join track using(album_id)
	join invoice_line using(track_id)
	group by artist.artist_id, artist.name
	order by sales desc
	limit 1
)

select customer_id, first_name, last_name, CTE.name, sum(invoice_line.unit_price * invoice_line.quantity) as Totall
from customer
join invoice using(customer_id)
join invoice_line using(invoice_id)
join track using(track_id)
join album using(album_id)
join CTE using(artist_id)
group by customer_id, first_name, last_name, CTE.name
order by Totall desc

In [ ]:
#Answer - Using CTE to get the sum as sales and then calling it in next query to get the required output.

### Question 10 - We want to find out the most popular music Genre for each country. We determine the most popular genre as the genre with the highest amount of purchases. Write a query that returns each country along with the top Genre. For countries where the maximum number of purchases is shared return all Genres.


In [ ]:
WITH CTE AS (
select billing_country, genre.name, count(quantity),
	row_number() over (partition by billing_country order by count(quantity) desc) as RN
from invoice
join invoice_line using(invoice_id)
join track using(track_id)
join genre using(genre_id)
group by 1, 2
order by  billing_country asc, count desc
	)
select * from CTE where RN = 1

### Question 11 - Write a query that determines the customer that has spent the most on music for each country. Write a query that returns the country along with the top customer and how much they spent. For countries where the top amount spent is shared, provide all customers who spent this amount.


In [ ]:
WITH CTE AS (
select first_name, last_name, country, sum(total),
	row_number() over(partition by country order by sum(total) desc) as RN
from customer
	join invoice using(customer_id)
	group by 1, 2, 3
	order by country asc, sum(total) desc
)
select * from CTE where RN = 1